In [169]:
from urllib import request
from bs4 import BeautifulSoup

def BBSThread(threadId, title, author, postTime, link):
    dic = {
        'threadId': threadId,
        'title': title,
        'author': author,
        'postTime': postTime,
        'link': link
    }
    return dic

def PrintThread(dic):
    return '[%s] / %s / by <%s> (%s)' % (dic['threadId'], dic['title'], dic['author'], dic['postTime'])


In [171]:
import datetime
import time

class Crawler:
    
    proxy_handler = request.ProxyHandler({
        'http':'http://127.0.0.1:8080',
        'https':'https://127.0.0.1.8080'
    })
    opener = request.build_opener(proxy_handler)
    request.install_opener(opener)

    head = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.94 Safari/537.36'
    }

    def __init__(self, delay):
        self.delay = delay
        self.last_accessed = datetime.datetime.now()

    def getUrl(self, url):
        
        if self.delay > 0:
            sleep_secs = self.delay - (datetime.datetime.now() - self.last_accessed).seconds

            if sleep_secs > 0:
                time.sleep(sleep_secs) 
        self.last_accessed = datetime.datetime.now()
        
        req = request.Request(url, headers=head)
        response = request.urlopen(req)

        html = response.read().decode('gbk')
        return html
    
    def bbsdoc(self, root, blink, page):
        html = self.getUrl(root + blink % page)

        soup = BeautifulSoup(html)
        objs = soup.select('tbody[id^=stickthread_]')
        objs += soup.select('tbody[id^=normalthread_]')
        threads = []
        for t in objs:
            title = t.select('th span[id^=thread_] a')[0].text
            link = t.select('th span[id^=thread_] a')[0]['href']
            threadId = link.split('-')[1]
            author = t.select('td.author cite a')[0].text
            postTime = t.select('td.author em')[0].text
            threads.append(BBSThread(threadId, title, author, postTime, link))
        return threads
    
    def bbscon(self, root, clink):
        html = self.getUrl(root + clink)
        
        soup = BeautifulSoup(html)
        postobj = soup.select('div[id^=postmessage_] div[id^=postmessage_]')[0]
        [x.decompose() for x in postobj.select('strong')]
        [x.decompose() for x in postobj.select('table')]
        return postobj.text

In [172]:
bbsroot = 'http://www.sis001.com/forum/'
blink = 'forum-383-%d.html'

In [173]:
crawler = Crawler(10)

In [174]:
threads = []
for i in range(1, 5):
    url = url_pattern % i
    threads += crawler.bbsdoc(bbsroot, blink, i)
    print('Get [%s] OK' % url)

Get [http://www.sis001.com/forum/forum-383-1.html] OK
Get [http://www.sis001.com/forum/forum-383-2.html] OK
Get [http://www.sis001.com/forum/forum-383-3.html] OK
Get [http://www.sis001.com/forum/forum-383-4.html] OK


In [175]:
import re
import os

white_list = ['琼明神女录', '册母为后', '绿帽武林之淫乱后宫', '龙珠肏', '锦绣江山传']

save_path = 'E:/turboc/sis'

all_tags = {}

for i in range(len(threads)):
    tags = re.findall('【(.*?)】', threads[i]['title'])
    for tag in tags:
        if not tag.isdigit():
            if not tag in all_tags:
                all_tags[tag] = []
            all_tags[tag].append(i)

for wtag in white_list:
    if wtag in all_tags:
        t = threads[all_tags[wtag][0]]
        chapter = crawler.bbscon(bbsroot, t['link'])
        txtpath = os.path.join(save_path, t['threadId']+'.txt')
        with open(txtpath, 'w', encoding='utf-8') as f:
            f.write(chapter)


In [176]:
import json

meta_data_path = os.path.join(save_path, 'meta.json')
with open(meta_data_path, 'w') as f:
    json.dump(threads, f)

In [177]:
threads

[{'threadId': '10272162',
  'title': '【勋章商店】庆第一会所七周年 绝版“SIS001七週歲紀念章”亲民价出售',
  'author': '亂花',
  'postTime': '2018-9-15',
  'link': 'thread-10272162-1-1.html'},
 {'threadId': '10264203',
  'title': '第一会所7周年暨庆中秋、迎国庆之『爱上会所』系列活动【9.16－10.16】',
  'author': '爱',
  'postTime': '2018-9-5',
  'link': 'thread-10264203-1-1.html'},
 {'threadId': '10260509',
  'title': '关于文区部分原创作者建群、收费的版务处理规则公告',
  'author': '尘埃',
  'postTime': '2018-8-30',
  'link': 'thread-10260509-1-1.html'},
 {'threadId': '10172251',
  'title': '找回论坛方法大全&如何防范被盗号&使用微信小程序进行二次验证',
  'author': '爱',
  'postTime': '2017-11-27',
  'link': 'thread-10172251-1-1.html'},
 {'threadId': '10202579',
  'title': '【色城★2018文心雕龙——欲目春情★征文大赛】正式开始啦*^_^*',
  'author': '缘心',
  'postTime': '2018-6-5',
  'link': 'thread-10202579-1-1.html'},
 {'threadId': '9858851',
  'title': '為解決盜號問題，即日起使用二次驗證（2FA）（2018年3月17日重要更新）',
  'author': '獅王',
  'postTime': '2017-2-19',
  'link': 'thread-9858851-1-1.html'},
 {'threadId': '10262779',
  'title': '色城2018◎欲目春情征文◎评委介